In [ ]:
# Create an instance of Candidate that implements the xValue property and
# initialise the game with your implementation

from holdet.campaign import Game, BaseCandidate


class CandidateEMA(BaseCandidate):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def xGrowth(self, alpha: float) -> float:
        """
        Predict the growth for the next round using exponential moving average
        (EMA). Set alpha to adjust the smoothing factor, between 0 and 1. Higher
        values give more weight to recent stats.
        """
        ema = 0.0
        for i, round in enumerate(sorted(self.rounds)):
            if i == 0:
                ema = round.xGrowth
            else:
                ema = alpha * round.xGrowth + (1 - alpha) * ema
        return ema

    @property
    def xValue(self) -> float:
        if self.captain:
            return self.value + self.xGrowth(0.5) * 2
        return self.value + self.xGrowth(0.5)


game = Game.new(CandidateEMA)


In [ ]:
# Set your budget and calculate the starting 11 that will maximize the expected
# value (xValue) within the rules of the game.

from rich.jupyter import print
from holdet.solver import lp
from holdet.formation import Formation

budget = 50 * 1000000  # 50 million

solution = lp.find_optimal_team(game.candidates, budget)
print(Formation(solution))
